In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from efficientnet_pytorch import EfficientNet
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

### Device and Transforms

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

### Load ResNet18

In [ ]:
resnet = models.resnet18(pretrained=False)
resnet.fc = nn.Sequential(
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 2)
)
resnet.load_state_dict(torch.load("resnet_facial_hair_binary.pt"))
resnet.to(device).eval()

### Load EfficientNet-B0

In [ ]:
effnet = EfficientNet.from_name('efficientnet-b0')
effnet._fc = nn.Linear(effnet._fc.in_features, 2)
effnet.load_state_dict(torch.load("efficientnet_facial.pt"))
effnet.to(device).eval()

### Ensemble Prediction Function

In [ ]:
def predict_ensemble(img_path, alpha=0.4):
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        resnet_out = torch.softmax(resnet(img_tensor), dim=1)
        effnet_out = torch.softmax(effnet(img_tensor), dim=1)

        # Weighted soft voting
        combined = (1 - alpha) * resnet_out + alpha * effnet_out
        _, pred = torch.max(combined, 1)
        return pred.item(), resnet_out.cpu().numpy(), effnet_out.cpu().numpy(), combined.cpu().numpy()

### Run Ensemble on Test Set

In [ ]:
test_dir = "processed_dataset/Test"
classes = ["No Beard", "Beard"]

results = []
for label in classes:
    label_dir = os.path.join(test_dir, label)
    true_label = 0 if label == "No Beard" else 1
    for fname in os.listdir(label_dir):
        fpath = os.path.join(label_dir, fname)
        pred, resnet_probs, effnet_probs, ensemble_probs = predict_ensemble(fpath, alpha=0.6)

        results.append({
            "filename": fname,
            "true_label": true_label,
            "resnet_pred": int(np.argmax(resnet_probs)),
            "effnet_pred": int(np.argmax(effnet_probs)),
            "ensemble_pred": pred,
            "ensemble_conf_beard": float(ensemble_probs[0][1])
        })

### Save and Analyze

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("ensemble_predictions.csv", index=False)
print("Saved results to ensemble_predictions.csv")

### Evaluation

In [ ]:
from sklearn.metrics import accuracy_score

print("\nClassification Report - Ensemble")
print(classification_report(results_df.true_label, results_df.ensemble_pred, target_names=classes))

cm = confusion_matrix(results_df.true_label, results_df.ensemble_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.title("Ensemble Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()